# Modelo RS_Envolvente
Si estamos ejecutando esta libreta desde Tellurium Notebook dejar activada la opción de Tellurium que se muestra a continuación.
En caso de usar Jupyter o notebooks similares desactivar para instalar la libreria.
#### AJUSTES

In [1]:
# Indicar si se esta ejecutando desde Tellurium
Tellurium = False
#===========================================================
# LOCALIZACION -> Indicar localizacion (ficheros adjuntos)
Localizacion = 'Albacete'
#===========================================================
# MINIMOS Y MAXIMOS DE CADA ALEATORIO
# Diario (Para aumentar o disminuir radiación base)
MinimoDiario = 0.8
MaximoDiario = 1.15
# Horario (Para simular nubes)
MinimoHorario = 0.75
MaximoHorario = 1
#===========================================================
# VARIABLE A CONSULTAR
especie = "H2O2" 
Todas_Las_Especies = True
#===========================================================
listaEspecies = ['APX', 'APXi', 'ASC', 'CoI', 'CoII', 'DHA', 'DHAR', 'GR', 'GSH', 'GSSG', 'MDA',  'NADP', 'NADPH', 'O2neg', 'SOD', 'TRXox', 'TRXred', 'PRXox', 'PRXred', 'H2O2', 'H2O', 'ECBBox', 'ECBBred']
listaVelocidades = ['vGR','vDHAR','vSOD','vAPX','v_APXH','v_APXMDA','kN','vFN','vF11','vF12','vF13','vF14']

## Implementación del modelo

In [2]:
if Tellurium == False:
    %pip install tellurium
#===========================================================
import math
import shutil
import random
import numpy as np
import tellurium as te
from datetime import datetime
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
#===========================================================

###### Nota
De momento se consigue la envolvente de la radiación leyendo los datos del PVGIS, se puede modificar de manera sencilla para dar nosotros 5 puntos de radiación para crear dicha envolvente

In [3]:
ficheroReferencia = "Referencias/TMY_"+Localizacion+".txt"
print('Localizacion:',Localizacion,'\n----------------------------------------------')
nPuntos = 12
RReferenciaAux = []
RReferencia = []
w = 2*math.pi/8760
dr = math.pi/180
hrs = np.arange(0,8760,0.1)

file = open(ficheroReferencia,'r')
lineas = file.readlines()
contLinea = 0
longitud = 0
for linea in lineas:
    contLinea += 1 
    if contLinea == 1:
        a,b = linea.split(":")
        latitud = float(b)
        print('Latitud:',latitud)
        
    if contLinea == 2:
        a,b = linea.split(":")
        longitud = float(b)
        print('Longitud:',longitud)
    
    if contLinea >= 18  and contLinea <= 8777:
        a,b,c,d,e,f,g,h,i,j = linea.split(",")
        RReferenciaAux.append(float(d))
file.close

difUTC = math.floor((longitud+7.5)/15)
print(difUTC)
RReferenciaAux = np.roll(RReferenciaAux,difUTC)
x1 = np.arange(0,8760.1,8760/nPuntos)
print('Puntos dia:', x1)
RReferenciaAux = np.roll(RReferenciaAux,240) # Roll para comenzar el 21 de diciembre

In [4]:
fotoperiodo = []
for i in x1:
    dia = i/24
    sd = 23.45*dr*math.sin((2*math.pi*(dia+274))/365) 
    #sd para modelos que empiecen el 1 de enero -> 23.45*dr*math.sin((2*math.pi*(dia+284))/365)
    ws = math.acos(-math.tan(sd)*math.tan(latitud*dr))
    wts = ws*(180/(math.pi*15))
    Tr = 12 - wts
    Ts = 12 + wts
    HorasDeSol = Ts - Tr
    fotoperiodo.append(HorasDeSol)
print('Horas de sol (fotoperiodo)\n--------------------------\n',fotoperiodo,'\n')

In [5]:
#-----------------------------------------------------------------------------
#AJUSTE DE y3
#-----------------------------------------------------------------------------
from matplotlib import pyplot as plt
import numpy as np
from scipy import interpolate

def ajustey3(y12,x1):
    print(len(y12))
    print(y12)
    print(len(x1))
    tck = interpolate.splrep(x1, y12, s=0, k=3)
    x_new = np.linspace(min(x1), max(x1), 87600)
    y_fit = interpolate.BSpline(*tck)(x_new)
    return y_fit

In [6]:
y3 = []
lista = RReferenciaAux.tolist()

mediaRAD = []
print('Punto 0')
for j in range(0,240,24):
    mediaRAD.append(np.max(lista[j:j+240]))
mediaRAD_MAX = np.max(mediaRAD)
print('Max Radiacion:',mediaRAD_MAX)
contMedia = 0
mediaRAD_calculo = 0
for j in range(10):
        if mediaRAD[j] >= mediaRAD_MAX*0.75:
            mediaRAD_calculo += mediaRAD[j]
            contMedia += 1
media = mediaRAD_calculo / contMedia
print('Media Radiacion:',media,'\n')
y3.append(media)

for i in range(0+int(8760/nPuntos),8760,int(8760/nPuntos)):
    print('Punto',i)
    mediaRAD = []
    for j in range(i,i+240,24):
        mediaRAD.append(np.max(lista[j:j+240]))

    for j in range(i,i-240,-24):
        mediaRAD.append(np.max(lista[j:j+240]))

    mediaRAD_MAX = np.max(mediaRAD)
    print('Max Radiacion:',mediaRAD_MAX)
    contMedia = 0
    mediaRAD_calculo = 0
    for j in range(20):
            if mediaRAD[j] >= mediaRAD_MAX*0.75:
                mediaRAD_calculo += mediaRAD[j]
                contMedia += 1
    media = mediaRAD_calculo / contMedia
    print('Media Radiacion:',media,'\n')
    y3.append(media)

mediaRAD = []
for j in range(8760-240,8760,24):
    mediaRAD.append(np.max(lista[j:j+240]))
mediaRAD_MAX = np.max(mediaRAD)
print('Max Radiacion:',mediaRAD_MAX)
contMedia = 0
mediaRAD_calculo = 0
for j in range(10):
        if mediaRAD[j] >= mediaRAD_MAX*0.75:
            mediaRAD_calculo += mediaRAD[j]
            contMedia += 1
media = mediaRAD_calculo / contMedia
print('Media Radiacion:',media,'\n')
y3.append(media)

print('y3\n--------------------------\n',y3,'\n')
plt.plot(x1,y3)

In [7]:
envolvente = ajustey3(y3,x1)
plt.plot(x1,y3,'*',hrs,envolvente)

In [8]:
for i in range(0,87600,int(87600/365)):
    dia = (i/240)+1
    sd = 23.45*dr*math.sin((2*math.pi*(dia+274))/365) 
    ws = math.acos(-math.tan(sd)*math.tan(latitud*dr))
    wts = ws*(180/(math.pi*15))
    Tr = 12 - wts
    Ts = 12 + wts
    #print('TR:',Tr,'TS',Ts,'DURACION =',Ts-Tr)

In [11]:
a1Values = []
for i in range(0,87600,int(87600/365)):
    a1Values.append(envolvente[i+120])
print(len(a1Values))

In [12]:
yRS = []
dr = math.pi/180
rango = np.arange(0.1,8760.1,0.1)
for time in rango:
    dia = math.ceil(time/24);
    sd = 23.45*dr*math.sin((2*math.pi*(dia+274))/365) 
    ws = math.acos(-math.tan(sd)*math.tan(latitud*dr))
    wts = ws*(180/(math.pi*15))
    sr = 12-wts;
    ss = 12+wts;
    ld = ss-sr;
    
    b1 = math.pi/ld 
    c1 = (-math.pi/ld)*sr
    
    if time <= ss+24*(dia-1) and time >= sr+24*(dia-1):
        yRS.append(a1Values[dia-1]*math.sin(b1*(time-24*(dia-1))+c1))
    else:
        yRS.append(0)
        

In [13]:
print('GENERANDO MODELO FINAL...')
rangoTime = np.arange(0,8761,1)
fsalida = open('Modelo/parte2.txt', 'w')
fsalida.write('\nlatitud = %f\n'%(latitud))
for i in range(0,87600,int(87600/365)):
    a1Values.append(envolvente[i+120])
    randomDiario = random.uniform(MinimoDiario,MaximoDiario)
    fsalida.write('at (dia == %d): a1 = %f, randomDiario = %f\n'%((i/240)+1,envolvente[i+120],randomDiario))
    if((i/240)+1 == 1):
        fsalida.write('a1 = %f #NO BORRAR -> Hace falta duplicarlo para inicializar a1\n'%(envolvente[i+120]))
for j in rangoTime:
    randomHorario = random.uniform(MinimoHorario,MaximoHorario)
    fsalida.write('at (time >= %d && time <%d): randomHorario = %f\n'%(j,j+1,randomHorario))
fsalida.write('end\n')
fsalida.close()
print('TERMINADO')

In [14]:
print('UNIENDO MODELO...')
with open('Modelo/MODELO_FINAL.txt','wb') as wfd:
    for f in ['Modelo/parte1Fija.txt','Modelo/parte2.txt']:
        with open(f,'rb') as fd:
            shutil.copyfileobj(fd, wfd)
print('TERMINADO')

In [15]:
import time
tic = time.perf_counter()
print('Cargando fichero...')
model = te.loada('Modelo/MODELO_FINAL.txt')
toc = time.perf_counter()
print(f'Fichero cargado en {toc - tic:0.2f} segundos')

In [16]:
n = 8760
print('Realizando simulaciones, espere por favor...')
tic = time.perf_counter()
model.resetAll()
if Todas_Las_Especies == True:
    simulation_result_especies = model.simulate(0.1,n,10*n+1)
    model.plot(simulation_result_especies)

    model.resetAll()
    simulation_result_F1 = model.simulate(0.1,n,10*n+1,["time",'F1RAD'])
    model.plot(simulation_result_F1)
    
else:
    simulation_result = model.simulate(0.1,n,10*n+1,["time",especie])

toc = time.perf_counter()
print(f'Simulacion ejecutada en {toc - tic:0.2f} segundos')

In [20]:
import pandas as pd

date = datetime.now()
fichero='ResultadosEspecies%s.txt'%date.strftime("%Y-%m-%d(%H%M%S)")
with open(fichero,'w', encoding='utf-8') as f:
    f.write(str(simulation_result_especies))
f.close()
with open(fichero,'r', encoding='utf-8') as f:
    data = f.read()
    data = data.replace("[", "")
    data = data.replace("]", "")
    data = data.replace(" ", "")
    data = data.replace(",\n", "\n")
    with open(fichero,'w', encoding='utf-8') as f2:
        f2.write(data)
        
fichero='ResultadosRADIACION%s.txt'%date.strftime("%Y-%m-%d(%H%M%S)")
with open(fichero,'w', encoding='utf-8') as f:
    f.write(str(simulation_result_F1))
f.close()
with open(fichero,'r', encoding='utf-8') as f:
    data = f.read()
    data = data.replace("[", "")
    data = data.replace("]", "")
    data = data.replace(" ", "")
    data = data.replace(",\n", "\n")
    with open(fichero,'w', encoding='utf-8') as f2:
        f2.write(data)